In [1]:
import face_recognition
from PIL import Image, ImageFile
import dlib
import numpy as np

In [2]:
#얼굴이미지를 로드한다.
face_image_np = face_recognition.load_image_file("img/soja.png")

In [3]:
# 얼굴을 추출한다.
face_locations = face_recognition.face_locations(face_image_np)

In [4]:
#얼굴 랜드마크를 추출한다.
face_landmarks = face_recognition.face_landmarks(face_image_np, face_locations)

In [5]:
face_img = Image.fromarray(face_image_np)
mask_img = Image.open("mask_image/mask1.png")
#face_img.show()
#mask_img.show()

In [6]:
for face_landmark in face_landmarks:
    print("test")

test


In [7]:
#콧등
bottom_lip = face_landmark['bottom_lip']
print("아래 입술 좌표 개수 : ",bottom_lip)
bottom_lip = bottom_lip[len(bottom_lip) * 1 // 12]
print("아래 입술 평균 좌표 : ",bottom_lip)
bottom_lip_v = np.array(bottom_lip)

아래 입술 좌표 개수 :  [(110, 98), (105, 104), (99, 106), (96, 106), (92, 106), (87, 104), (80, 100), (83, 100), (91, 100), (95, 100), (99, 100), (108, 99)]
아래 입술 평균 좌표 :  (105, 104)


In [8]:
#턱
chin = face_landmark['chin']
print(face_landmark['chin'])
chin_len = len(chin)
print("턱 좌표 개수 : ",chin_len)

chin_bottom_point = chin[chin_len // 2]
print("턱 하관 좌표 : ",chin_bottom_point)
chin_bottom_v = np.array(chin_bottom_point)

print("목 좌표 : ",chin_bottom_v)
chin_left_point = chin[chin_len // 8]
print("턱 왼쪽 좌표 : ",chin_left_point)

chin_right_point = chin[chin_len * 7 // 8]
print("턱 오른쪽 좌표 : ",chin_right_point)

[(56, 65), (57, 74), (59, 84), (61, 92), (66, 100), (71, 107), (79, 114), (87, 119), (96, 120), (104, 119), (112, 114), (119, 107), (124, 100), (128, 91), (130, 81), (130, 71), (130, 61)]
턱 좌표 개수 :  17
턱 하관 좌표 :  (96, 120)
목 좌표 :  [ 96 120]
턱 왼쪽 좌표 :  (59, 84)
턱 오른쪽 좌표 :  (130, 81)


In [9]:
# split mask and resize
width = mask_img.width
height = mask_img.height
width_ratio = 0.9
new_height = int(np.linalg.norm(bottom_lip_v - chin_bottom_v))
print("아래 입술y-턱하관y : ",new_height)

아래 입술y-턱하관y :  18


In [10]:
def get_distance_from_point_to_line(point, line_point1, line_point2):
        distance = np.abs( (line_point2[1] - line_point1[1]) * point[0] +
                          (line_point1[0] - line_point2[0]) * point[1] +
                          (line_point2[0] - line_point1[0]) * line_point1[1] +
                          (line_point1[1] - line_point2[1]) * line_point1[0]) / \
                   np.sqrt((line_point2[1] - line_point1[1]) * (line_point2[1] - line_point1[1]) +
                           (line_point1[0] - line_point2[0]) * (line_point1[0] - line_point2[0]))
        return int(distance)

In [11]:
# left
mask_left_img = mask_img.crop((0, 0, width // 2, height))

#mask_left_img.show()
mask_left_width = get_distance_from_point_to_line(chin_left_point, bottom_lip, chin_bottom_point)

print("왼쪽 마스크 너비:",mask_left_width)
mask_left_width = int(mask_left_width * width_ratio)
print("비율 왼쪽 마스크 너비 : ",mask_left_width)
mask_left_img = mask_left_img.resize((mask_left_width, new_height))
#mask_left_img.show()

왼쪽 마스크 너비: 49
비율 왼쪽 마스크 너비 :  44


In [12]:
# right
mask_right_img = mask_img.crop((width // 2, 0, width, height))
mask_right_width = get_distance_from_point_to_line(chin_right_point, bottom_lip, chin_bottom_point)

print("오른쪽 마스크 너비 :",mask_right_width)
mask_right_width = int(mask_right_width * width_ratio)
print("비율 오른쪽 마스크 너비 :",mask_right_width)
mask_right_img = mask_right_img.resize((mask_right_width, new_height))

오른쪽 마스크 너비 : 10
비율 오른쪽 마스크 너비 : 9


In [13]:
# merge mask
size = (mask_left_img.width + mask_right_img.width, new_height)
print("size : ",size)
mask_img = Image.new('RGBA', size)
mask_img.paste(mask_left_img, (0, 0), mask_left_img)
mask_img.paste(mask_right_img, (mask_left_img.width, 0), mask_right_img)
#mask_img.show()

size :  (53, 18)


In [14]:
# rotate mask
angle = np.arctan2(chin_bottom_point[1] - bottom_lip[1], chin_bottom_point[0] - bottom_lip[0])
rotated_mask_img = mask_img.rotate(angle, expand=True)
#rotated_mask_img.show()

In [15]:
# calculate mask location
center_x = (bottom_lip[0] + chin_bottom_point[0]) // 2
print("center_x, : ", center_x)
center_y = (bottom_lip[1] + chin_bottom_point[1]) // 2
print("center_y, : ", center_y)

center_x, :  100
center_y, :  112


In [16]:
offset = mask_img.width // 2 - mask_left_img.width
print("offset : ", offset)
radian = angle * np.pi / 180
print("radian : ", radian)
box_x = center_x + int(offset * np.cos(radian)) - rotated_mask_img.width // 2
print("box_x : ", box_x)
box_y = center_y + int(offset * np.sin(radian)) - rotated_mask_img.height // 2
print("box_y : ", box_y)

offset :  -18
radian :  0.036358450915742954
box_x :  56
box_y :  102


In [17]:
# add mask
box_x = box_x+10
box_y = box_y+5
face_img.paste(mask_img, (box_x, box_y), mask_img)
merge_face_mask:ImageFile = None
merge_face_mask=face_img
merge_face_mask.show()